<a href="https://colab.research.google.com/github/JoseMD12/finance-inter/blob/main/Financeiro_Inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bem-vindo a Análise Financeira!

Este notebook foi criado para ajudar a organizar e analisar seus extratos bancários.

**Primeiros Passos:**

1.  **Suba seu arquivo CSV:** Clique no ícone de pasta no painel lateral esquerdo. Em seguida, clique no ícone "Upload para armazenamento da sessão" (um ícone de seta para cima).

2.  **Modifique `nome_do_arquivo` e `arquivo_saida`:**
    *   `nome_do_arquivo`: Altere o valor entre as aspas para o nome exato do arquivo CSV que você acabou de subir.

    *   `arquivo_saida`: Defina o nome que você deseja dar ao arquivo CSV.

Após configurar esses passos, você pode executar as células do notebook para processar seus dados.

In [42]:
import pandas as pd
from IPython import get_ipython
from IPython.display import display

#Colocar o nome do arquivo aqui
nome_do_arquivo = "/Extrato-10-05-2025-a-14-06-2025.csv"
#Colocar o nome do arquivo saida
arquivo_saida = "/Junho-Jose.csv"

df = pd.read_csv(nome_do_arquivo, skiprows=5, sep=';')
df.columns = ["Data", "Descricao", "Valor", "Saldo"]
df = df.drop(columns=["Saldo"])

## Limpeza da Coluna 'Descricao'

Este bloco de código utiliza expressões regulares para limpar a coluna 'Descricao', removendo termos indesejados como "pix enviado:", "pix recebido:", etc. Isso ajuda a padronizar as descrições para facilitar a categorização posterior.

In [43]:
import re

def limpar_descricao(desc):
    padroes = [
        r'(?i)^pix enviado:\s*"?',
        r'(?i)^pix recebido:\s*"?',
        r'(?i)^pagamento de convenio:\s*"?',
        r'(?i)^compra no debito:\s*"?no estabelecimento\s*"?',
        r'(?i)^pagamento efetuado:\s*"?'
    ]

    for padrao in padroes:
        desc = re.sub(padrao, '', desc)

    desc = desc.replace('"', '').strip()
    return desc

df['Descricao'] = df['Descricao'].astype(str).apply(limpar_descricao)

## Categorização das Transações

Este bloco de código categoriza as transações com base em palavras-chave presentes na coluna 'Descricao'.

In [44]:
categorias = {
    "Mercado": ["cestto", "market4u", "center shop", "hortifruti", "padaria"],
    "Contas": ["realize", "claro", "ceee", "banco do brasil"],
    "Casa": ["amazon"],
    "Saude": ["panvel", "fernanda", "amor e saude", "convenio"],
    "Lazer": ["bar", "ingresso", "silva"],
    "Carro": ["posto", "combustivel", "oficina", "estacionamento"],
    "Mel": ["cobasi"],
    "Trabalho": ["creatus", "pnb", "preto no branco", "erenito xavier", "paralela"],
    "Delivery": ["ifood"],
    "Assinatura": ["max", "gzh", "spotify"]
}

def categorizar_descricao(desc):
    desc_lower = str(desc).lower()
    for categoria, palavras_chave in categorias.items():
        for palavra_chave in palavras_chave:
            if palavra_chave in desc_lower:
                return categoria
    return ""

df['Categoria'] = df['Descricao'].apply(categorizar_descricao)

In [48]:

def classificar_tipo_gasto(categoria):
    categorias_individual = ["Lazer", "Investimento", "Saúde", "Pagamento", "Trabalho"]
    categorias_conjunto = ["Mercado", "Contas", "Carro", "Mel", "Casa", "Roupas", "Delivery", "Assinatura"] # Incluí as novas categorias que você adicionou

    if categoria in categorias_individual:
        return "Individual"
    elif categoria in categorias_conjunto:
        return "Conjunto"
    else:
        return ""

df['Tipo do Gasto'] = df['Categoria'].apply(classificar_tipo_gasto)

## Formatação Final do DataFrame e Exportação

Este bloco de código prepara o DataFrame final para exportação. Ele seleciona as colunas relevantes, renomeia-as para facilitar a leitura e adiciona uma coluna vazia 'Tipo do Gasto'. Finalmente, o DataFrame formatado é exportado para um arquivo CSV com o nome especificado na variável `arquivo_saida`, usando ponto-vírgula como separador.

In [49]:
df_formatado = df[[
    "Descricao",
    "Data",
    "Valor",
    "Categoria",
    "Tipo do Gasto",
]]

df_formatado.columns = [
    "Descrição do Gasto",
    "Data",
    "Valor da transação",
    "Categoria",
    "Tipo do Gasto",
]

df_formatado.head()
df_formatado.to_csv(arquivo_saida, index=False, sep=';')